In [16]:
# Imports!
import sys
import os
import numpy as np
from skimage.feature import hog
from skimage.transform import resize
from skimage import io, data, color, exposure
import pandas as pd
import csv
import cv2
import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np

In [17]:
data = pd.read_csv("../train_photo_to_biz_ids.csv")

In [21]:
# train_img = [os.listdir("../train_photos/")]
# image_root = "../train_photos/"
# C:\Users\Shreyas\Desktop\RestaurantClassification\train_photos\train_photos
train_img = [os.listdir("../train_photos/")]
image_root = "../train_photos/"

In [22]:
# Returns greyscale and resized image
def get_image(path):
    img = cv2.imread(path)
    
    if img is not None:
        img_resize = cv2.resize(img, (299,299))
        return img_resize
    return None

In [23]:
# https://code.oursky.com/tensorflow-svm-image-classifications-engine/
def create_graph(model_path):
    """
    create_graph loads the inception model to memory, should be called before
    calling extract_features.
 
    model_path: path to inception model in protobuf form.
    """
    with gfile.FastGFile(model_path, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

create_graph("../Inception V3/tensorflow_inception_graph.pb")

In [24]:
# Extracting the bottleneck feature using CNN 

def get_feature(image_path):
    feature_dimension = 2048
    with tf.Session() as sess:
            flattened_tensor = sess.graph.get_tensor_by_name('pool_3:0')
            ph = tf.placeholder(tf.string, shape=[])
            image_data = get_image(image_path)
            image_data = np.array(image_data)
            image_data = np.expand_dims(image_data,axis=0)
            feature = sess.run(flattened_tensor, {
                'Mul:0':  image_data
            })
            return np.squeeze(feature)
        

In [25]:
# Extracting features in batches. This should be faster
def get_feature_from_list(image_path_list):
    feature_dimension = 2048
    features = np.empty((len(image_path_list), feature_dimension))

    with tf.Session() as sess:
            flattened_tensor = sess.graph.get_tensor_by_name('pool_3:0')
            for i, image_path in enumerate(image_path_list):
                image_data = get_image(image_path)
                image_data = np.array(image_data)
                image_data = np.expand_dims(image_data,axis=0)
                feature = sess.run(flattened_tensor, {
                    'Mul:0':  image_data
                })
                features[i, :] = np.squeeze(feature)
                
    return features 

In [26]:
# Utility to print count
def printCount(cnt):
    if cnt==10:
        print("10 images done!")
    elif cnt ==100:
        print("100 images done!")
    elif cnt ==500:
        print("500 images done!")
    elif cnt == 1000:
        print("1000 images done!")
    elif cnt%10000 == 0:
        print(str(cnt)+" Images Done!")

In [27]:
# Extract feature for all images
img_features = []
batch_size = 10000
# Start processing the images
with open(r"../train_data/train_feature_CNN.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter =",",quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["imageid", "features"])
    
    image_path_list = []
    image_id_list = []
    cnt = 0
    for imageid in data['photo_id']:
        img_path = image_root + str(imageid) + ".jpg"
        image_path_list.append(img_path)
        image_id_list.append(imageid)
        # Start batch processing
        if(len(image_path_list) == batch_size):
            img_features = get_feature_from_list(image_path_list)
            for image_id, feature in enumerate(img_features):
                writer.writerow([image_id_list[image_id], feature.tolist()])
            
            cnt+=batch_size
            printCount(cnt)
            image_id_list = []
            image_path_list = []
#     If some more images are left
    if(len(image_path_list)> 0):
            img_features = get_feature_from_list(image_path_list)
            #print(img_features.shape)
            for image_id, feature in enumerate(img_features):
                #print("writing imageid"+ str(image_id_list[image_id]))
                writer.writerow([image_id_list[image_id], feature.tolist()])

10000 Images Done!
20000 Images Done!
30000 Images Done!
40000 Images Done!
50000 Images Done!
60000 Images Done!
70000 Images Done!
80000 Images Done!
90000 Images Done!
100000 Images Done!
110000 Images Done!
120000 Images Done!
130000 Images Done!
140000 Images Done!
150000 Images Done!
160000 Images Done!
170000 Images Done!
180000 Images Done!
190000 Images Done!
200000 Images Done!
210000 Images Done!
220000 Images Done!
230000 Images Done!
